# Scales

PyLabRobot supports the following scales:

- Mettler Toledo WXS205SDU

Scales are controlled by the {class}`~pylabrobot.scales.scale.Scale` class. This class takes a backend as an argument. The backend is responsible for communicating with the scale and is specific to the hardware being used.

In [1]:
from pylabrobot.scales import Scale
from pylabrobot.scales.mettler_toledo import MettlerToledoWXS205SDU

In [2]:
backend = MettlerToledoWXS205SDU(port="/dev/cu.usbserial-110")  # take any ScaleBackend you want
scale = Scale(backend=backend, size_x=0, size_y=0, size_z=0)
await scale.setup()

The {meth}`~pylabrobot.scales.scale.Scale.setup` method is used to initialize the scale. This is where the backend will connect to the scale and perform any necessary initialization.

The {class}`~pylabrobot.scales.scale.Scale` class has a number of methods for controlling the scale and reading measurements. These are:

- {meth}`~pylabrobot.scales.scale.Scale.get_weight`: Read the weight from the scale in grams.
- {meth}`~pylabrobot.scales.scale.Scale.tare`: Tare the scale.
- {meth}`~pylabrobot.scales.scale.Scale.zero`: Zero the scale.

=(MettlerToledoWXS205SDU)

## Mettler Toledo WXS205SDU

The Mettler Toledo XS205 scale is controlled by the {class}`~pylabrobot.scales.mettler_toledo_backend.MettlerToledoWXS205SDU` class. This scale is used by the Hamilton Liquid Verification Kit (LVK).

The scale comes with an RS-232 serial port. You'll probably want to use a USB to serial adapter to connect it to your computer. Any $10 generic USB to serial adapter should work (e.g. something that uses FTDI).

Note that this scale has a 'warm-up' time after plugging in that is documented as 60-90 minutes by Mettler Toledo depending on the document you like at. In our experience, 30 minutes is sufficient. If you try to take a measurement before this time, you will likely get a "Command understood but currently not executable (balance is currently executing another command)" error. Sometimes plugging the power cord in and out will make things work faster.

In [3]:
backend = MettlerToledoWXS205SDU(port="/dev/cu.usbserial-110")
scale = Scale(backend=backend)
await scale.setup()
await scale.get_weight(timeout="stable")

0.00148

This scale provides various timeouts:

- `"stable"`: The time to wait for the scale to stabilize before reading the weight. Note that this may take a very long time to finish if the scale cannot take a stable reading. If you're not using the air enclosure, even being near the scale can cause it to never stabilize.
- 0: Read the value immediately
- n>0: Try to get a stable value for n seconds. If the value is stable before n seconds, return it immediately. Otherwise, return the value after n seconds.

These parameters are available for {meth}`~pylabrobot.scales.mettler_toledo.MettlerToledoWXS205SDU.get_weight`, {meth}`~pylabrobot.scales.mettler_toledo.MettlerToledoWXS205SDU.tare`, and {meth}`~pylabrobot.scales.mettler_toledo.MettlerToledoWXS205SDU.zero`.

In [4]:
await scale.get_weight(timeout=0)

0.00148

## Example: getting timing information

Let's say you wanted to determine how long it takes to take a measurement. In PyLabRobot, this is easy:

In [5]:
backend = MettlerToledoWXS205SDU(port="/dev/cu.usbserial-110")
s = Scale(backend=backend)
await s.setup()

In [6]:
import time
import numpy as np

l = []
for i in range(10):
  t0 = time.monotonic_ns()
  await scale.get_weight(timeout="stable")
  t1 = time.monotonic_ns()
  l.append((t1 - t0) / 1e6)

print(f"{np.mean(l):.2f} ms ± {np.std(l):.2f} ms")

100.44 ms ± 6.78 ms
